In [1]:
# Libraries
import os
import requests
import json
import urllib
import fnmatch
import folium
import pandas as pd
from IPython.display import clear_output
from pandas.io.json import json_normalize


# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
url = 'http://overpass-api.de/api/interpreter'

# Overpass turbo query
query = f"""
        [out:json];
        area["ISO3166-1"="CH"][admin_level=2];
        node ["shop"="car"](area);
        out;"""


# Web API request
r = requests.get(url, params={'data': query})
data = r.json()['elements']

# Save data to file
with open('carvendors.json', 'w') as json_file:
    json.dump(data, json_file)

# Store data in data frame
df = json_normalize(data)
df.head(5)

,type,id,lat,lon,tags.addr:city,tags.addr:housenumber,tags.addr:postcode,tags.addr:street,tags.brand,tags.brand:wikidata,...,tags.operator:website,tags.office,tags.payment:cards,tags.payment:invoice,tags.service:vehicle:Sale,tags.opening_hours:signed,tags.addr:full,tags.payment:coins,tags.payment:postfinance_card,tags.mobile
0,node,56699071,47.234366,8.735632,Stäfa,155,8712,Seestrasse,Škoda,Q29637,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,node,112017810,47.350203,8.527605,Zürich,4,8038,Thujastrasse,Abarth;Alfa Romeo;Fiat;Lancia,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,node,246658345,47.229552,8.962142,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,node,259589969,47.029684,8.294093,NaN,NaN,NaN,NaN,Toyota,Q53268,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,node,274592776,47.489136,7.595573,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Subset of supermarkets by brand
loc2= df[["lat", "lon", "tags.brand", "tags.addr:housenumber","tags.addr:street","tags.name"]]

print(loc2)



           lat       lon                     tags.brand tags.addr:housenumber  \
0    47.234366  8.735632                          Škoda                   155   
1    47.350203  8.527605  Abarth;Alfa Romeo;Fiat;Lancia                     4   
2    47.229552  8.962142                            NaN                   NaN   
3    47.029684  8.294093                         Toyota                   NaN   
4    47.489136  7.595573                            NaN                   NaN   
..         ...       ...                            ...                   ...   
942  47.255395  7.818477                            NaN                     3   
943  46.801457  7.119741                        Peugeot                   NaN   
944  47.091639  6.811776                            NaN                   NaN   
945  46.589044  7.084241             Mitsubishi;Hyundai                   NaN   
946  47.229561  8.843061                            NaN                   NaN   

    tags.addr:street       

In [9]:
# Remove duplicates
loc2 = loc2.drop_duplicates()

# Remove missing values
loc2 = loc2.dropna()

print(loc2)

           lat       lon                     tags.brand tags.addr:housenumber  \
0    47.234366  8.735632                          Škoda                   155   
1    47.350203  8.527605  Abarth;Alfa Romeo;Fiat;Lancia                     4   
34   47.174636  7.540673                      Le Garage                    32   
37   47.190774  7.553083                          Volvo                     1   
50   47.040624  7.624709                        Peugeot                     1   
..         ...       ...                            ...                   ...   
913  47.377901  8.511583                  Renault;Dacia                   330   
920  46.248107  6.955989               Kia, Opel, Isuzu                    34   
921  46.955450  8.334331                        Peugeot                     4   
925  47.373732  8.537498                       Polestar                     9   
933  47.641201  9.199856            Opel Hyundai Suzuki                    89   

      tags.addr:street     

In [5]:
import sqlite3
import fnmatch
import os

# Create data base
conn = sqlite3.connect('vendors.db')
cursor = conn.cursor()

# Show dbs in the directory
flist = fnmatch.filter(os.listdir('.'), '*.db')
for i in flist:
    print(i)

vendors.db


In [6]:
#create SQL table in the database

cursor.execute('''CREATE TABLE IF NOT EXISTS vendors_table (lat DECIMAL(2,13),
                                                                lon DECIMAL(2,13),
                                                                Brand VARCHAR(100),
                                                                Housenumber INT(4),
                                                                Street VARCHAR(50),
                                                                Vendorname VARCHAR(50))''')
# Confirm changes to the table
conn.commit()

In [7]:
#Write data to the SQL-table in data base
loc2.to_sql(name = 'vendors_table',
            con = conn,
         index = False,
         if_exists = 'replace')

128

In [ ]:
# information to query data from sqlite is found in SW2/apartments_database_Python.html

In [8]:

# Create map
map = folium.Map(loc2=[loc2.lat.mean(), 
                           loc2.lon.mean()], 
                 zoom_start=8, 
                 control_scale=True)

# Add maker symbols
for index, loc2_info in loc2.iterrows():
    folium.Marker([loc2_info["lat"], 
                   loc2_info["lon"]], 
                  popup=loc2_info["tags.name"]).add_to(map)

# Plot map
map
